In [1]:
import numpy as np
import pandas as pd
from src.algorithms import * 
from src.utils import * 
from src.metrics_FAMD import *

In [2]:
gbsg = pd.read_csv("../GBSG2")

# On a un jeu de données complet: 
np.where(gbsg.isna()==True)

(array([], dtype=int64), array([], dtype=int64))

On détermine $I$ et $J$: 

In [4]:
I, J = gbsg.shape

Trouver $K_1$, $K_2$

In [5]:
# Categorical Variables :
idx_k2_gbsg = pd.Index(["meno","hormon","status", "grade"])

# Continuous Variables
idx_k1_gbsg = gbsg.columns.difference(idx_k2_gbsg)

In [6]:
# Inject missing values into the gbsg dataframe :
proba_non_missing = 0.8
gbsg_missing = create_missingness(gbsg, proba_non_missing)

# Encode dummy variables in gbsg dataframe and in the gbsg dataframe with missing values :
gbsg_missing_dummy, idx_j_gbsg, nb_values_per_cat_gbsg = encode_dummy_variables(gbsg_missing, idx_k2_gbsg)
gbsg_dummy = encode_dummy_variables(gbsg, idx_k2_gbsg)[0]

gbsg_missing_dummy

,pid,age,size,nodes,pgr,er,rfstime,meno_1,meno_0,hormon_0,hormon_1,status_0,status_1,grade_3,grade_2,grade_1
0,132,NaN,18,NaN,0,0,1838,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN
1,NaN,55,20,16,0,NaN,403,1,0,NaN,NaN,0,1,1,0,0
2,1140,56,40,NaN,0,0,1603,1,0,1,0,NaN,NaN,1,0,0
3,NaN,45,25,1,0,4,177,0,1,1,0,1,0,1,0,0
4,130,65,NaN,NaN,0,36,1855,1,0,0,1,NaN,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,NaN,51,30,2,1152,38,1760,0,1,0,1,1,0,1,0,0
682,1273,64,26,2,1356,NaN,1152,1,0,0,1,1,0,0,1,0
683,1525,57,35,1,1490,209,NaN,1,0,0,1,1,0,1,0,0
684,736,44,NaN,3,NaN,NaN,629,0,1,1,0,NaN,NaN,0,1,0


In [7]:
#Check the proportion of missing data :
n_missing = gbsg_missing.isna().sum().sum()
n_missing/(gbsg_missing.shape[0]*gbsg_missing.shape[1])

0.19666048237476808

In [8]:
#FAMD :
famd_algo = FAMD(data=gbsg_missing_dummy, k1=idx_k1_gbsg, k2=idx_j_gbsg)
#famd_algo.run_famd()

In [9]:
#IFAMD :

n_it = 1000
ifamd_gbsg = IterativeFAMDImputer(n_components=4, data=gbsg_missing_dummy, k1=idx_k1_gbsg, k2=idx_j_gbsg, nb_values_per_cat = nb_values_per_cat_gbsg)
ifamd_gbsg.impute(n_it)

df = ifamd_gbsg.df

Maximum iterations reached


#### To ensure that we are only considering TPR and NRMSE scores on imputed data: 

In [10]:
C0_missing = gbsg_missing_dummy.isna()[ifamd_gbsg.k1].to_numpy()
Categ_missing = gbsg_missing_dummy.isna()[ifamd_gbsg.k2].to_numpy()

#### Falsely classified Rate: 

In [11]:
# We encode categories into 0,1
res = (df[ifamd_gbsg.k2].copy()>=0.5).astype(int)

per_categ = metric_fc(res[Categ_missing], gbsg_dummy[ifamd_gbsg.k2][Categ_missing])
rate = per_categ.mean()

In [12]:
rate

0.13617444806369888

#### NRMSE

In [13]:
compute_nrmse_weighted(df[ifamd_gbsg.k1][C0_missing].astype(int), gbsg_dummy[ifamd_gbsg.k1][C0_missing])

0.06906736285409001